In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from lib import model_device_io as io
from lib import data_loader as dl
from lib import models as md

In [2]:
trainset = dl.HotelReservationData(root='data/train')
trainset_loader = DataLoader(trainset, batch_size=4096, shuffle=True, num_workers=0)
device = io.getCudaDevice(cudaNum = 1, torchSeed = 123)
model = md.Classifier(input_size = 258).to(device)

Device used: cuda:1
